In [ ]:
import random
import typing

import numpy as np

from pylib import microsoro

np.random.seed(1)
random.seed(1)


In [ ]:
microsoro.defaults.k = 1e4
microsoro.defaults.b = 50
microsoro.defaults.b_lim = 10, 100
microsoro.defaults.dt = 0.001
microsoro.defaults.k_lim = (1e4, 5e4)
microsoro.defaults.m_lim = (1, 10)
microsoro.defaults.l_lim = (0.2, 1.5)

style = microsoro.Style()
style.ylim = (-5, 10)
style.time_dilation = 2
style.scale = 20


def evaluate_structure(
    structure: microsoro.Structure, viz: bool = True
) -> float:
    return (
        microsoro.perform_simulation(
            setup_regimen_conditioners=[
                microsoro.conditioners.ApplyTranslate(dpx=3, dpy=4),
                microsoro.conditioners.ApplyPropel(dvx=-10, dvy=-2),
                microsoro.conditioners.ApplySpin(360.0),
                microsoro.conditioners.ApplyRotate(18),
            ],
            update_regimen_components=[
                *microsoro.get_default_update_regimen(structure=structure),
                microsoro.components.ApplyFloorBounce(m=-1, b=6),
                microsoro.components.EvaluateDuration(
                    microsoro.components.BundleComponents(
                        microsoro.components.HaltPastFinishLine(
                            b=-14.0,
                            m=1.0,
                            independent_axis="horizontal",
                            comparator=lambda a, b: np.all(np.less(a, b)),
                        ),
                        microsoro.components.HaltAfterElapsedTime(10.0),
                    ),
                ),
                microsoro.components.BundleComponents(
                    microsoro.components.PaceToWalltime(style=style),
                    # switch comment-out to save video instead of showing animation
                    microsoro.components.ShowAnimationIpycanvas(style=style),
                    # microsoro.components.SaveVideoPyglet("evolved.mp4", style=style),
                )
                if viz
                else microsoro.components.NopComponent(),
            ],
        ),
    )


In [ ]:
def evaluate(individual: np.ndarray, viz: bool = False) -> float:
    b, k, l, m = (individual[i, :, :] for i in range(4))

    structure = microsoro.Structure.make_from_bytes(b=b, k=k, l=l, m=m)
    try:
        return evaluate_structure(structure, viz=viz)
    except Exception as e:
        print(f"exception! {e}")
        return 1e9


# Make random structure and evaluate


In [ ]:
structure = microsoro.Structure.make_random(
    interpolate_from_height=4, interpolate_from_width=4
)
evaluate_structure(structure, viz=False)


In [ ]:
evaluate_structure(structure, viz=True)


# Evolve


In [ ]:
from multiprocessing import Pool
import random

from deap import base, creator, tools, algorithms
import numpy as np
from tqdm import tqdm


# Initialize DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # minimize t
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator for 4x4 2D array
def create_4x4x4_matrix():
    return np.random.randint(0, 256, (4, 4, 4))


toolbox.register("attr_matrix", create_4x4x4_matrix)

# Structure initializers
toolbox.register(
    "individual", tools.initIterate, creator.Individual, toolbox.attr_matrix
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operators
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=255, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Parallelism
pool = Pool(5)
toolbox.register("map", pool.map)

# Create an initial population of 300 individuals
population = toolbox.population(n=30)

# Number of generations
ngen = 10
# Probability with which two individuals are crossed
cxpb = 0.7
# Probability with which an individual is mutated
mutpb = 0.2

# Run the algorithm
algorithms.eaSimple(
    population,
    toolbox,
    cxpb,
    mutpb,
    ngen,
    stats=None,
    halloffame=None,
    verbose=True,
)

# Print the best individual after the evolution
fits = [ind.fitness.values[0] for ind in population]
length = len(population)
mean = sum(fits) / length
sum2 = sum(x * x for x in fits)
std = abs(sum2 / length - mean**2) ** 0.5
print("Min %s" % min(fits))
print("Max %s" % max(fits))
print("Avg %s" % mean)
print("Std %s" % std)

best_ind = tools.selBest(population, 1)[0]
print("Best individual: %s" % best_ind)


# Visualize Evolved Result


In [ ]:
evaluate(best_ind, viz=True)
